Project Description :


Classification is probably the most popular task that you would deal with in real life.  Text in the form of blogs, posts, articles, etc. is written every second. It is a challenge to predict the  information about the writer without knowing about him/her.     We are going to create a classifier that predicts multiple features of the author of a given text.  We have designed it as a Multilabel classification problem. 

Data set info :


Blog Authorship Corpus  Over 600,000 posts from more than 19 thousand bloggers    The Blog Authorship Corpus consists of the collected posts of 19,320 bloggers gathered from  blogger.com in August 2004. The corpus incorporates a total of 681,288 posts and over 140 million  words - or approximately 35 posts and 7250 words per person.    Each blog is presented as a separate file, the name of which indicates a blogger id# and the  blogger’s self-provided gender, age, industry, and astrological sign. (All are labeled for gender and  age but for many, industry and/or sign is marked as unknown.)    All bloggers included in the corpus fall into one of three age groups:  8240 "10s" blogs (ages 13-17),  8086 "20s" blogs(ages 23-27)  2994 "30s" blogs (ages 33-47) 

  For each age group, there is an equal number of male and female bloggers.  Each blog in the corpus includes at least 200 occurrences of common English words. All formatting  has been stripped with two exceptions. Individual posts within a single blogger are separated by the  date of the following post and links within a post are denoted by the label urllink.    Link to dataset:  https://www.kaggle.com/rtatman/blog-authorship-corpus/downloads

In [0]:
# imports
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report,f1_score, accuracy_score, recall_score, precision_score
from nltk.stem import WordNetLemmatizer

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


Step 1 :


Load the dataset (5 points)  a. Tip: As the dataset is large, use fewer rows. Check what is working well on your  machine and decide accordingly.

In [0]:
# read blogtext.csv into a DataFrame
df1 = pd.read_csv('gdrive/My Drive/AIML/NLP_assignment/blogtext.csv')

In [21]:
blogtext

,id,gender,age,topic,sign,date,text
0,2059027,male,15,Student,Leo,"14,May,2004",aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa...
1,2059027,male,15,Student,Leo,"13,May,2004",aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa...
2,2059027,male,15,Student,Leo,"12,May,2004",aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa...
3,2059027,male,15,Student,Leo,"12,May,2004",aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa
4,3581210,male,33,InvestmentBanking,Aquarius,"11,June,2004",aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa...
...,...,...,...,...,...,...,...
681279,1713845,male,23,Student,Taurus,"01,July,2004","Dear Susan, I could write some really ..."
681280,1713845,male,23,Student,Taurus,"01,July,2004","Dear Susan, 'I have the second yeast i..."
681281,1713845,male,23,Student,Taurus,"01,July,2004","Dear Susan, Your 'boyfriend' is fuckin..."
681282,1713845,male,23,Student,Taurus,"01,July,2004","Dear Susan: Just to clarify, I am as..."


In [24]:
df1

,id,gender,age,topic,sign,date,text
0,2059027,male,15,Student,Leo,"14,May,2004","Info has been found (+/- 100 pages,..."
1,2059027,male,15,Student,Leo,"13,May,2004",These are the team members: Drewe...
2,2059027,male,15,Student,Leo,"12,May,2004",In het kader van kernfusie op aarde...
3,2059027,male,15,Student,Leo,"12,May,2004",testing!!! testing!!!
4,3581210,male,33,InvestmentBanking,Aquarius,"11,June,2004",Thanks to Yahoo!'s Toolbar I can ...
...,...,...,...,...,...,...,...
681279,1713845,male,23,Student,Taurus,"01,July,2004","Dear Susan, I could write some really ..."
681280,1713845,male,23,Student,Taurus,"01,July,2004","Dear Susan, 'I have the second yeast i..."
681281,1713845,male,23,Student,Taurus,"01,July,2004","Dear Susan, Your 'boyfriend' is fuckin..."
681282,1713845,male,23,Student,Taurus,"01,July,2004","Dear Susan: Just to clarify, I am as..."


In [25]:
df1.shape

(681284, 7)

In [4]:
blogtext.shape

(681284, 7)

In [0]:
df = blogtext

In [6]:
df.shape

(681284, 7)

In [22]:
df.head(10)

,id,gender,age,topic,sign,date,text
0,2059027,male,15,Student,Leo,"14,May,2004",aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa...
1,2059027,male,15,Student,Leo,"13,May,2004",aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa...
2,2059027,male,15,Student,Leo,"12,May,2004",aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa...
3,2059027,male,15,Student,Leo,"12,May,2004",aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa
4,3581210,male,33,InvestmentBanking,Aquarius,"11,June,2004",aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa...
5,3581210,male,33,InvestmentBanking,Aquarius,"10,June,2004",aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa...
6,3581210,male,33,InvestmentBanking,Aquarius,"10,June,2004",aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa...
7,3581210,male,33,InvestmentBanking,Aquarius,"10,June,2004",aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa...
8,3581210,male,33,InvestmentBanking,Aquarius,"10,June,2004",aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa...
9,3581210,male,33,InvestmentBanking,Aquarius,"09,June,2004",aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa...


In [10]:
# Taking only initial 5k rows to initial pre processing & training
df_sample = df[:3000]
print(df_sample.shape)
df_sample["text"].loc[0]

(3000, 7)


'           Info has been found (+/- 100 pages, and 4.5 MB of .pdf files) Now i have to wait untill our team leader has processed it and learns html.         '

In [26]:
# Taking only initial 3k rows to initial pre processing & training, 
#Taking from the new dataset df2, as the replace function impacts the new dataset in the previous run
df2 = df1[:3000]
print(df2.shape)
df2["text"].loc[0]

(3000, 7)


'           Info has been found (+/- 100 pages, and 4.5 MB of .pdf files) Now i have to wait untill our team leader has processed it and learns html.         '

In [20]:
df.head(1)

,id,gender,age,topic,sign,date,text
0,2059027,male,15,Student,Leo,"14,May,2004",aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa...


In [27]:
df2.head(1)

,id,gender,age,topic,sign,date,text
0,2059027,male,15,Student,Leo,"14,May,2004","Info has been found (+/- 100 pages,..."


In [18]:
# Taking only initial 5k rows to initial pre processing & training
df_sample_1 = df[:3000]
print(df_sample_1.shape)
df_sample_1["text"].loc[0]

(3000, 7)


'aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa'

In [11]:
df_sample.dtypes


id         int64
gender    object
age        int64
topic     object
sign      object
date      object
text      object
dtype: object

In [28]:
df2.dtypes

id         int64
gender    object
age        int64
topic     object
sign      object
date      object
text      object
dtype: object

Step 2 :

Preprocess rows of the “text” column.

a. Remove unwanted characters  
b. Convert text to lowercase  
c. Remove unwanted spaces  
d. Remove stopwords 

In [17]:
#Removing unwanted / special characters
df_sample['text'] = df_sample['text'].str.replace('[^A-Za-z]',' ') #Removes special character from df_sample["text"].loc[0] - +/- 100,4.5 and . 
                                                                   #and keeps only the string characters from A-Z
df_sample["text"].loc[0]                                           #Note - This method changes in the core data/base dataset, hence repeated the first step
                                                                   #and created a new variable df1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


'aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa'

In [16]:
df_sample['text'] = df_sample['text'].str.replace('[^A-Za-z]','a') #Removes special character from df_sample["text"].loc[0] - +/- 100,4.5 and . 
                                                                   #and keeps only the string characters from A-Z
df_sample["text"].loc[0]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


'aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa'

In [29]:
#Removing unwanted / special characters
df2['text'] = df2['text'].str.replace('[^A-Za-z]',' ') #Removes special character from df_sample["text"].loc[0] - +/- 100,4.5 and . 
                                                                   #and keeps only the string characters from A-Z
df2["text"].loc[0]                                           #Note - This method changes in the core data/base dataset(blogspot),

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


'           Info has been found          pages  and     MB of  pdf files  Now i have to wait untill our team leader has processed it and learns html          '

In [30]:
# Coverting to lower case
df2['text'] = df2['text'].str.lower()
df2["text"].loc[0]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


'           info has been found          pages  and     mb of  pdf files  now i have to wait untill our team leader has processed it and learns html          '

In [31]:
#Removing spaces
df2["text"] = df2["text"].str.strip()  #what ever value u pass here will be converter into string format by the .str function
df2["text"].loc[0]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


'info has been found          pages  and     mb of  pdf files  now i have to wait untill our team leader has processed it and learns html'

In [32]:
df2["text"] = df2["text"].str.split()  # splitting each row of text data into individual words.
# So it can be iterated through to remove only stopwords in next steps.

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [37]:
df2.head(6) #here you can see the text column split into individual words

,id,gender,age,topic,sign,date,text
0,2059027,male,15,Student,Leo,"14,May,2004","[info, has, been, found, pages, and, mb, of, p..."
1,2059027,male,15,Student,Leo,"13,May,2004","[these, are, the, team, members, drewes, van, ..."
2,2059027,male,15,Student,Leo,"12,May,2004","[in, het, kader, van, kernfusie, op, aarde, ma..."
3,2059027,male,15,Student,Leo,"12,May,2004","[testing, testing]"
4,3581210,male,33,InvestmentBanking,Aquarius,"11,June,2004","[thanks, to, yahoo, s, toolbar, i, can, now, c..."
5,3581210,male,33,InvestmentBanking,Aquarius,"10,June,2004","[i, had, an, interesting, conversation, with, ..."


# Removing stop words





In [41]:
import nltk
nltk.download('stopwords') #Download the stopwords from nltk

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
stop = stopwords.words('english')             #Default method to remove stopwords
def removestopwords(y):   # Function definition
 stopwordremoved = [w for w in y if w not in stop]
 return(" ".join(stopwordremoved))

In [43]:
text_column_size = df2["text"].size
print("text column size :", text_column_size)

# Initialize an empty list to hold the text after stop word removal
df2_stop_words_removed = []

# Loop over each text
for i in range( 0, text_column_size):
    df2_stop_words_removed.append(removestopwords(df2["text"][i]))

text column size : 3000


In [46]:
df2_stop_words_removed[0]

'info found pages mb pdf files wait untill team leader processed learns html'

In [49]:
df2_stop_words_removed[10]

'ah korean language looks difficult first figure read hanguel korea surprisingly easy learn alphabet characters seems easy vocabulary starts oh backwards us sentence structure yikes luckily many options us slow witted foreigners take language course could list urllink joongang article says lot resources urllink well guy motivation jeon ji hyun latest something actually star movies cfs hear means commercial feature positive saw latest movie sunday night hard describe name english version windstruck korean version yeochinso short ne yeojachingu rul sogayhamnida like introduce girlfriend surprisingly titles make sense like website korean english looks quite good actually urllink movie shown theatres subtitles special times info urllink list many theatres seoul click urllink urllink great reason learn korean already married went foreigners well local korean national course korean take picture put urllink movie hof bar update bud mine passed urllink link giordano ad apparently aired korea n

In [50]:
#Replace text column with cleaner_corpus_df_sample_text 
df2["text"] = df2_stop_words_removed
df2["text"][10]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


'ah korean language looks difficult first figure read hanguel korea surprisingly easy learn alphabet characters seems easy vocabulary starts oh backwards us sentence structure yikes luckily many options us slow witted foreigners take language course could list urllink joongang article says lot resources urllink well guy motivation jeon ji hyun latest something actually star movies cfs hear means commercial feature positive saw latest movie sunday night hard describe name english version windstruck korean version yeochinso short ne yeojachingu rul sogayhamnida like introduce girlfriend surprisingly titles make sense like website korean english looks quite good actually urllink movie shown theatres subtitles special times info urllink list many theatres seoul click urllink urllink great reason learn korean already married went foreigners well local korean national course korean take picture put urllink movie hof bar update bud mine passed urllink link giordano ad apparently aired korea n

In [51]:
df2.head() #Stop words removed dataframe

,id,gender,age,topic,sign,date,text
0,2059027,male,15,Student,Leo,"14,May,2004",info found pages mb pdf files wait untill team...
1,2059027,male,15,Student,Leo,"13,May,2004",team members drewes van der laag urllink mail ...
2,2059027,male,15,Student,Leo,"12,May,2004",het kader van kernfusie op aarde maak je eigen...
3,2059027,male,15,Student,Leo,"12,May,2004",testing testing
4,3581210,male,33,InvestmentBanking,Aquarius,"11,June,2004",thanks yahoo toolbar capture urls popups means...


# **Lemmatization**

In [0]:
w_tokenizer = nltk.tokenize.WhitespaceTokenizer() #Default method for lemmatization
lemmatizer = nltk.stem.WordNetLemmatizer()

def lemmatize_text(text):
    lemm = [lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(text)]
    return(" ".join(lemm)) 

In [54]:
import nltk
nltk.download('wordnet') #download wordnet before lemmatization

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [55]:
df2["text"] = df2.text.apply(lemmatize_text)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [56]:
df2.head() #Pages in the first row in previous dataset is changed to page after lemmatization

,id,gender,age,topic,sign,date,text
0,2059027,male,15,Student,Leo,"14,May,2004",info found page mb pdf file wait untill team l...
1,2059027,male,15,Student,Leo,"13,May,2004",team member drewes van der laag urllink mail r...
2,2059027,male,15,Student,Leo,"12,May,2004",het kader van kernfusie op aarde maak je eigen...
3,2059027,male,15,Student,Leo,"12,May,2004",testing testing
4,3581210,male,33,InvestmentBanking,Aquarius,"11,June,2004",thanks yahoo toolbar capture url popups mean s...


In [57]:
df2["text"][10] # Lemmatized output

'ah korean language look difficult first figure read hanguel korea surprisingly easy learn alphabet character seems easy vocabulary start oh backwards u sentence structure yikes luckily many option u slow witted foreigner take language course could list urllink joongang article say lot resource urllink well guy motivation jeon ji hyun latest something actually star movie cf hear mean commercial feature positive saw latest movie sunday night hard describe name english version windstruck korean version yeochinso short ne yeojachingu rul sogayhamnida like introduce girlfriend surprisingly title make sense like website korean english look quite good actually urllink movie shown theatre subtitle special time info urllink list many theatre seoul click urllink urllink great reason learn korean already married went foreigner well local korean national course korean take picture put urllink movie hof bar update bud mine passed urllink link giordano ad apparently aired korea nothing xxx sensibil

# **Step 3:**
As this is a multi-label classification problem, merge  all the label columns together, so that we have all the labels together for a particular sentence. a. Label columns to merge: “gender”, “age”, “topic”, “sign”

In [58]:
#name of available columns 
df2.columns

Index(['id', 'gender', 'age', 'topic', 'sign', 'date', 'text'], dtype='object')

In [61]:
df2.dtypes #age is an interger column

id         int64
gender    object
age        int64
topic     object
sign      object
date      object
text      object
dtype: object

In [60]:
df2.shape

(3000, 7)

In [62]:
# merge gender', 'age', 'topic', 'sign'
df2['age'] = df2['age'].astype(str)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [63]:
df2.dtypes

id         int64
gender    object
age       object
topic     object
sign      object
date      object
text      object
dtype: object

In [64]:
df2['labels'] = df2[['gender','age','topic','sign']].apply(lambda x: ','.join(x), axis = 1) #merged using lambda function

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [65]:
df2

,id,gender,age,topic,sign,date,text,labels
0,2059027,male,15,Student,Leo,"14,May,2004",info found page mb pdf file wait untill team l...,"male,15,Student,Leo"
1,2059027,male,15,Student,Leo,"13,May,2004",team member drewes van der laag urllink mail r...,"male,15,Student,Leo"
2,2059027,male,15,Student,Leo,"12,May,2004",het kader van kernfusie op aarde maak je eigen...,"male,15,Student,Leo"
3,2059027,male,15,Student,Leo,"12,May,2004",testing testing,"male,15,Student,Leo"
4,3581210,male,33,InvestmentBanking,Aquarius,"11,June,2004",thanks yahoo toolbar capture url popups mean s...,"male,33,InvestmentBanking,Aquarius"
...,...,...,...,...,...,...,...,...
2995,589736,male,35,Technology,Aries,"05,August,2004",zoo exhibit thing mucho mucho funny,"male,35,Technology,Aries"
2996,589736,male,35,Technology,Aries,"05,August,2004",fave song day aimee mann pavlov bell album los...,"male,35,Technology,Aries"
2997,589736,male,35,Technology,Aries,"05,August,2004",urllink america best zoo exhibit,"male,35,Technology,Aries"
2998,589736,male,35,Technology,Aries,"05,August,2004",le one make declaritive statement le apt appea...,"male,35,Technology,Aries"


In [0]:
#Another method for merging
## As there are no null values ,we proceed with merging of gender,age,topic,sign into labels col  

#df2 = df2.assign(labels = df2.gender.astype(str) + ', ' + \
  # df2.age.astype(str) + ', ' + df2.topic.astype(str) + ', '+ \
  # df2.sign.astype(str))

In [66]:
df2_merged = df2.drop(labels = ['date','gender', 'age','topic','sign','id'], axis = 1)
df2_merged.head()

,text,labels
0,info found page mb pdf file wait untill team l...,"male,15,Student,Leo"
1,team member drewes van der laag urllink mail r...,"male,15,Student,Leo"
2,het kader van kernfusie op aarde maak je eigen...,"male,15,Student,Leo"
3,testing testing,"male,15,Student,Leo"
4,thanks yahoo toolbar capture url popups mean s...,"male,33,InvestmentBanking,Aquarius"


In [67]:
df2_merged.shape

(3000, 2)

# Step4:

Separate features and labels, and split the data into training and testing

In [0]:
feature = df2_merged['text']

In [0]:
df2_merged['labels'] = df2_merged['labels'].str.lower() #Converting the values in the lables column to lower case
labels = df2_merged['labels']

In [70]:
feature.head()

0    info found page mb pdf file wait untill team l...
1    team member drewes van der laag urllink mail r...
2    het kader van kernfusie op aarde maak je eigen...
3                                      testing testing
4    thanks yahoo toolbar capture url popups mean s...
Name: text, dtype: object

In [72]:
labels.head()

0                   male,15,student,leo
1                   male,15,student,leo
2                   male,15,student,leo
3                   male,15,student,leo
4    male,33,investmentbanking,aquarius
Name: labels, dtype: object

In [0]:
X_train, X_test, Y_train, Y_test = train_test_split(feature,labels, test_size = 0.33, random_state = 143)

In [75]:
print(X_train.shape)
print(Y_train.shape)

(2010,)
(2010,)


In [77]:
print((X_test.shape))
print(Y_test.shape)

(990,)
(990,)


# Step5 :

Vectorizing the features.

a. Create a Bag of Words using count vectorizer  

i. Use ngram_range=(1, 2)  
ii. Vectorize training and testing features 

b. Print the term-document matrix 

In [0]:
# Creating Bag of words
vectorizer = CountVectorizer(min_df = 2,ngram_range = (1,2),stop_words = "english")
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [81]:
X_train[2]

<1x16018 sparse matrix of type '<class 'numpy.int64'>'
	with 10 stored elements in Compressed Sparse Row format>


# Step 6:


Create a dictionary to get the count of every label i.e. the key will be label name and value will  be the total count of the label.



In [82]:
vectorizer_labels = CountVectorizer(min_df = 1,ngram_range = (1,1),stop_words = "english")
labels_vector = vectorizer_labels.fit_transform(labels)
vectorizer_labels.vocabulary_

{'14': 0,
 '15': 1,
 '16': 2,
 '17': 3,
 '23': 4,
 '24': 5,
 '25': 6,
 '26': 7,
 '27': 8,
 '33': 9,
 '34': 10,
 '35': 11,
 '37': 12,
 '39': 13,
 '41': 14,
 '44': 15,
 '45': 16,
 'accounting': 17,
 'aquarius': 18,
 'aries': 19,
 'arts': 20,
 'banking': 21,
 'businessservices': 22,
 'cancer': 23,
 'capricorn': 24,
 'communications': 25,
 'education': 26,
 'engineering': 27,
 'female': 28,
 'gemini': 29,
 'indunk': 30,
 'internet': 31,
 'investmentbanking': 32,
 'leo': 33,
 'libra': 34,
 'libraries': 35,
 'male': 36,
 'media': 37,
 'museums': 38,
 'non': 39,
 'pisces': 40,
 'profit': 41,
 'recreation': 42,
 'sagittarius': 43,
 'science': 44,
 'scorpio': 45,
 'sports': 46,
 'student': 47,
 'taurus': 48,
 'technology': 49,
 'virgo': 50}

In [83]:
vectorizer_labels.vocabulary_.keys() 

dict_keys(['male', '15', 'student', 'leo', '33', 'investmentbanking', 'aquarius', 'female', '14', 'indunk', 'aries', '25', 'capricorn', '17', 'gemini', '23', 'non', 'profit', 'cancer', 'banking', '37', 'sagittarius', '26', '24', 'scorpio', '27', 'education', '45', 'engineering', 'libra', 'science', '34', '41', 'communications', 'media', 'businessservices', 'sports', 'recreation', 'virgo', 'taurus', 'arts', 'pisces', '44', '16', 'internet', 'museums', 'libraries', 'accounting', '39', '35', 'technology'])

In [86]:
sorted(vectorizer_labels.vocabulary_.keys())

['14',
 '15',
 '16',
 '17',
 '23',
 '24',
 '25',
 '26',
 '27',
 '33',
 '34',
 '35',
 '37',
 '39',
 '41',
 '44',
 '45',
 'accounting',
 'aquarius',
 'aries',
 'arts',
 'banking',
 'businessservices',
 'cancer',
 'capricorn',
 'communications',
 'education',
 'engineering',
 'female',
 'gemini',
 'indunk',
 'internet',
 'investmentbanking',
 'leo',
 'libra',
 'libraries',
 'male',
 'media',
 'museums',
 'non',
 'pisces',
 'profit',
 'recreation',
 'sagittarius',
 'science',
 'scorpio',
 'sports',
 'student',
 'taurus',
 'technology',
 'virgo']

In [87]:
#fitting these abover values into a variable as classes
# Extracing only key value from above dictionary, which contains unique labels. These set of labels will be used as classes in 
# multilabelbinariser further.
label_classes = []  
for key in vectorizer_labels.vocabulary_.keys():
    label_classes.append(key)
    
print(sorted(label_classes))

['14', '15', '16', '17', '23', '24', '25', '26', '27', '33', '34', '35', '37', '39', '41', '44', '45', 'accounting', 'aquarius', 'aries', 'arts', 'banking', 'businessservices', 'cancer', 'capricorn', 'communications', 'education', 'engineering', 'female', 'gemini', 'indunk', 'internet', 'investmentbanking', 'leo', 'libra', 'libraries', 'male', 'media', 'museums', 'non', 'pisces', 'profit', 'recreation', 'sagittarius', 'science', 'scorpio', 'sports', 'student', 'taurus', 'technology', 'virgo']


Step 7:


Transform the labels - As we have noticed before, in this task each example can have multiple tags. To deal with  such kind of prediction, we need to transform labels in a binary form and the prediction will be  a mask of 0s and 1s. For this purpose, it is convenient to use ​MultiLabelBinarizer​ from sklearn 

a. Convert your train and test labels using MultiLabelBinarizer

In [0]:
mlb = MultiLabelBinarizer(classes = label_classes)  # initialising multilabelbinariser with all unique possible classes

In [0]:
# transform to dictionary
y_train_2 = [set(i.split(',')) for i in Y_train]
y_test_2 = [set(i.split(',')) for i in Y_test]

In [90]:
y_train_2[0]

{'35', 'aries', 'male', 'technology'}

In [91]:
y_test_2[0]

{'27', 'aquarius', 'education', 'female'}

In [0]:
mlb = MultiLabelBinarizer()
y_train = mlb.fit_transform(y_train_2)
y_test = mlb.transform(y_test_2)

In [98]:
y_train[0]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0])

In [99]:
y_test[0]

array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0])

In [0]:
mlb = MultiLabelBinarizer()
y_train_2 = mlb.fit_transform(y_train)
y_test_2 = mlb.transform(y_test)

In [96]:
y_train_2[0]

array([1, 1])

In [97]:
y_test_2[0]

array([1, 1])

In [104]:
y_test[0]

array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0])

In [105]:
y_test

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 1, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 1, 0]])

In [103]:
y_train[0]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0])

In [106]:
y_train

array([[0, 0, 0, ..., 0, 1, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 1, 0],
       ...,
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [102]:
mlb.classes_ #this is because y_test and y_train is transformed 2 times

array([0, 1], dtype=object)

In [118]:
len(mlb.classes_)

2

Step 8:


In this task, we suggest using the One-vs-Rest approach, which is implemented in  OneVsRestClassifier​ class. In this approach k classifiers (= number of tags) are trained. As a  basic classifier, use ​LogisticRegression​. It is one of the simplest methods, but often it  performs good enough in text classification tasks. It might take some time because the  number of classifiers to train is large. 

a. Use a linear classifier of your choice, wrap it up in OneVsRestClassifier to train it on  every label 

In [0]:
clf = LogisticRegression(solver = 'lbfgs',max_iter = 1000)  # initiating the classifier
#from sklearn.svm import SVC
#clf = SVC(kernel = "linear")
clf = OneVsRestClassifier(clf)

Step 9:

Fit the classifier, make predictions and get the accuracy

a. Print the following  
i. Accuracy score  
ii. F1 score  
iii. Average precision score  
iv. Average recall score 

Tip: Make sure you are familiar with all of them. How would you expect the  things to work for the multi-label scenario? Read about micro/macro/weighted  averaging 

In [0]:
clf.fit(X_train, y_train)
y_pred_class = clf.predict(X_test)

In [110]:
 from sklearn import metrics
 print("Accuracy score------------",metrics.accuracy_score(y_test, y_pred_class))
 print("F1 score micro------------",metrics.f1_score(y_test, y_pred_class,average='micro'))
 print("F1 score macro------------",metrics.f1_score(y_test, y_pred_class,average='macro'))
 print("Average precision score------------",metrics.precision_score(y_test, y_pred_class,average='weighted'))
 print("Average Recall score------------",metrics.recall_score(y_test, y_pred_class,average='weighted'))

Accuracy score------------ 0.5686868686868687
F1 score micro------------ 0.7558299039780522
F1 score macro------------ 0.3200567621004007
Average precision score------------ 0.7859473177412591
Average Recall score------------ 0.6957070707070707


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


10.Print true label and predicted label for any five examples (7.5 points)

In [114]:
Y_pred_inv = mlb.inverse_transform(y_pred_class)   # inverse transforming predited label data
#_test_trans_inv =  mlb.inverse_transform(y_test) # inverse transforming original test label data

ValueError: ignored

In [117]:
y_test_pred_inversed = mlb.inverse_transform(y_pred_class) #this error is because the y_test is transformed 2 times,and its array size has become 1,2
y_test_inversed = mlb.inverse_transform(y_test)
for i in range(10,15):
    print('Title:\t{}\nTrue labels:\t{}\nPredicted labels:\t{}\n\n'.format(
        X_test.get(i),
        ','.join(y_test_inversed[i]),
        ','.join(y_test_pred_inversed[i])
    ))

ValueError: ignored